<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 2: Practical - LDA & QDA <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Arshia Soltani Moakhar
    
____

### Full Name : Erfan Sadraiye
### Student Number : 99101835
___
https://colab.research.google.com/drive/1PF1Ze4L13d7U-xDhLZ4pm8Wy7JvDGK3G?usp=sharing

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as SKLinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as SKQuadraticDiscriminantAnalysis
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, recall_score, f1_score

# Exploration (20 points)

In [2]:
!wget https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv

--2023-04-14 12:11:08--  https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv [following]
--2023-04-14 12:11:09--  https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89401 (87K) [text/plain]
Saving to: ‘dataset-train.csv.1’

dataset-train.csv.1 100%[===================>]  87.31K  --.-KB/s    in 0.02s   

2023-04-14 12:11:09 (4.92 MB/s) - ‘dataset-train.c

Load the dataset as a dataframe

In [3]:
training_dataset = pd.read_csv('dataset-train.csv')
training_dataset.head(5)

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Timestamp,Clicked on Ad,Gender
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,Tunisia,2016-03-27 00:53:11,0,female
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,Nauru,2016-04-04 01:39:02,0,male
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,San Marino,2016-03-13 20:35:42,0,female
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,Italy,2016-01-10 02:31:19,0,male
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,Iceland,2016-06-03 03:36:18,0,female


Get the minimum "Area Income" for all Italians

In [4]:
min_italian_area_income = training_dataset[training_dataset['Country'] == 'Italy']['Area Income'].min()
min_italian_area_income

54286.1

Make a new dataframe called "sample_df" by sampling 100 rows from the previously made dataframe

In [5]:
sample_df = training_dataset.sample(n=100)
sample_df.head(5)

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Timestamp,Clicked on Ad,Gender
61,85.84,34,64654.66,192.93,Front-line heuristic data-warehouse,Lake Annashire,Saint Vincent and the Grenadines,2016-06-15 05:30:13,0,male
682,43.57,36,50971.73,125.20,Front-line upward-trending groupware,Lake Joshuafurt,French Polynesia,2016-06-25 17:33:35,1,male
102,89.91,40,59593.56,194.23,Innovative background conglomeration,Wallacechester,Maldives,2016-07-09 14:55:36,0,female
139,75.80,38,70575.60,146.19,Progressive clear-thinking open architecture,Ashleychester,Thailand,2016-04-18 11:23:05,0,male
235,40.42,45,40183.75,133.90,Stand-alone reciprocal synergy,Lisamouth,Norway,2016-03-16 07:59:37,1,male


Compare all men and women aged less than 30 years in terms of their average "Daily Internet Usage". Your result should contain the average daily usage for each age and gender, seperately.

In [6]:
training_dataset[training_dataset['Age'] < 30].groupby(['Age','Gender'])['Daily Internet Usage'].mean()

Age  Gender
19   female    184.940000
     male      189.523333
20   female    202.160000
     male      198.982500
21   female    186.980000
     male      212.815000
22   female    200.070000
     male      188.837500
23   female    187.105714
     male      184.615000
24   female    178.693333
     male      200.055556
25   female    193.618571
     male      206.527857
26   female    192.741875
     male      203.916250
27   female    189.157692
     male      205.916154
28   female    205.873333
     male      196.416842
29   female    191.191200
     male      202.005714
Name: Daily Internet Usage, dtype: float64

# Preprocessing (30 points)

Remove the following columns: "Ad Topic Line", "Timestamp"

In [7]:
modified_dataset = training_dataset.drop(['Ad Topic Line','Timestamp'], axis=1 )
modified_dataset.head(5)

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,City,Country,Clicked on Ad,Gender
0,68.95,35,61833.90,256.09,Wrightburgh,Tunisia,0,female
1,80.23,31,68441.85,193.77,West Jodi,Nauru,0,male
2,69.47,26,59785.94,236.50,Davidton,San Marino,0,female
3,74.15,29,54806.18,245.89,West Terrifurt,Italy,0,male
4,68.37,35,73889.99,225.58,South Manuel,Iceland,0,female


Convert the "City" and "Country" columns to their one-hot encoded versions.
For each set of one-hot encoded columns, remove one of them to make the columns not correlated. Finally, remove the original columns named "City" and "Country".

In [8]:
dataframe_with_city_country_one_hot = pd.get_dummies(modified_dataset, columns=['City','Country'], drop_first=True, dtype=int)

In [9]:
dataframe_with_city_country_one_hot.head(5)

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad,Gender,City_Alanview,City_Alexanderfurt,City_Alexanderview,City_Alexisland,...,Country_Uruguay,Country_Uzbekistan,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia,Country_Zimbabwe
0,68.95,35,61833.90,256.09,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,69.47,26,59785.94,236.50,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,68.37,35,73889.99,225.58,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Convert "Gender" column to a new binary column (having only 0 and 1 values). Then remove the original column.

In [10]:
from sklearn.preprocessing import LabelEncoder

dataframe_with_binary_gender = dataframe_with_city_country_one_hot.copy()
le = LabelEncoder()
dataframe_with_binary_gender['Gender'] = le.fit_transform(dataframe_with_city_country_one_hot['Gender'])
dataframe_with_binary_gender.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad,Gender,City_Alanview,City_Alexanderfurt,City_Alexanderview,City_Alexisland,...,Country_Uruguay,Country_Uzbekistan,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia,Country_Zimbabwe
0,68.95,35,61833.90,256.09,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,69.47,26,59785.94,236.50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,68.37,35,73889.99,225.58,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


By dropping the "Clicked on Ad" column, make a new dataframe called "features_df"

In [11]:
features_df = dataframe_with_binary_gender.drop(['Clicked on Ad'], axis=1)

Make a new dataframe called "target_df" containing only the "Clicked on Ad" column

In [12]:
target_df = dataframe_with_binary_gender['Clicked on Ad']

Make two numpy arrays called "x" and "y" from "features_df" and "target_df" dataframes, respectively.

In [13]:
x = features_df.to_numpy()
y = target_df.to_numpy()

# Training Models (40 points)

Split the dataset (`x` and `y`) into "training" and "testing" parts containing 80% and 20% of the data respectively.

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Train both LDA and QDA models on the data using `sklearn`

In [15]:
sklda = SKLinearDiscriminantAnalysis()

In [16]:
sklda.fit(x_train, y_train)

LinearDiscriminantAnalysis()

In [17]:
skqda = SKQuadraticDiscriminantAnalysis()

In [18]:
skqda.fit(x_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

Train LDA and QDA using your own code!

Linear Discriminant Analysis

In [19]:
class LinearDiscriminantAnalysis:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self, alpha=0.0):
        self.priors = None
        self.means = None
        self.cov = None
        self.classes = None
        self.inv_cov = None
        self.alpha = alpha

    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        y : array-like, shape = [n_samples, n_labels]
            Labels
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        self.priors = np.zeros(n_classes)
        self.means = np.zeros((n_classes, n_features))
        self.cov = np.zeros((n_features, n_features), dtype=np.float64)
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.means[i, :] = X_c.mean(axis=0)
            self.priors[i] = 1.0 * X_c.shape[0] / n_samples

        X_centered = X - np.mean(X, axis=0)
        _, s, v = np.linalg.svd(X_centered, full_matrices=False)
        s = s ** 2 / (n_samples - 1)
        s = ((1 - self.alpha) * s) + self.alpha
        self.cov = np.matmul(s * v.T, v)
        # self.cov = np.cov(X.T)
        self.inv_cov = np.linalg.pinv(self.cov)

    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        Returns
        -------
        Predicted value
        """
        n_samples, n_features = X.shape
        n_classes = len(self.classes)
        inv_cov = self.inv_cov
        posterior = np.zeros((n_samples, n_classes))
        for k in range(n_classes):
            mean = self.means[k, :]
            prior = np.log(self.priors[k])
            for i in range(n_samples):
                x = X[i, :]
                posterior[i, k] = np.matmul(np.matmul(x.T, inv_cov).T, mean) - 0.5 * np.matmul(
                    np.matmul(mean.T, inv_cov).T,
                    mean) + prior
        y_pred = self.classes[np.argmax(posterior, axis=1)]
        return y_pred

Quadratic Discriminant Analysis (Optional) (15 bonus points)

In [20]:
class QuadraticDiscriminantAnalysis:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self, alpha=0.0):
        self.priors = None
        self.means = None
        self.cov = None
        self.inv_cov = None
        self.classes = None
        self.log_dets = None
        self.alpha = alpha

    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        y : array-like, shape = [n_samples, n_labels]
            Labels
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        self.priors = np.zeros(n_classes)
        self.means = np.zeros((n_classes, n_features))
        self.cov = np.zeros((n_classes, n_features, n_features), dtype=np.float64)
        self.inv_cov = np.zeros((n_classes, n_features, n_features), dtype=np.float64)
        self.log_dets = np.zeros(n_classes, dtype=np.float64)
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.means[i, :] = X_c.mean(axis=0)
            self.priors[i] = 1.0 * X_c.shape[0] / n_samples
            self.cov[i] = np.cov(X_c.T)
            X_centered = X_c - np.mean(X_c, axis=0)
            _, s, v = np.linalg.svd(X_centered, full_matrices=False)
            s = s ** 2 / (n_samples - 1)
            s = ((1 - self.alpha) * s) + self.alpha
            self.cov[i] = np.matmul(s * v.T, v)
            self.log_dets[i] = np.sum(np.log(s))
            self.inv_cov[i] = np.linalg.pinv(self.cov[i])

    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        Returns
        -------
        Predicted value
        """
        n_samples, n_features = X.shape
        n_classes = len(self.classes)
        posterior = np.zeros((n_samples, n_classes))
        for k in range(n_classes):
            inv_cov = self.inv_cov[k]
            prior = np.log(self.priors[k])
            log_det = -0.5 * self.log_dets[k]
            mean_k = self.means[k, :]
            for i in range(n_samples):
                x_centered = X[i, :] - mean_k
                posterior[i, k] = -0.5 * np.matmul(np.matmul(x_centered.T, inv_cov).T, x_centered) + prior + log_det
        y_pred = self.classes[np.argmax(posterior, axis=1)]
        return y_pred

In [21]:
lda = LinearDiscriminantAnalysis(0.01)
lda.fit(x_train,y_train)

qda = QuadraticDiscriminantAnalysis(0.01)
qda.fit(x_train,y_train)

# Evaluation (10 points)

Using all 4 models, predict the labels for all samples belonging to the "testing" part

In [22]:
y_pred_sklda = sklda.predict(x_test)
y_pred_skqda = skqda.predict(x_test)
y_pred_lda = lda.predict(x_test)
y_pred_qda = qda.predict(x_test)

Calculate Accuracy, Recall, and F1-Scores

In [23]:
print("Sklearn :")
print(f"Accuracy Score for LDA: ", accuracy_score(y_test, y_pred_sklda))
print(f"Recall Score for LDA: ", recall_score(y_test, y_pred_sklda))
print(f"F1 Score for LDA: ", f1_score(y_test, y_pred_sklda))
print(classification_report(y_test, y_pred_sklda))
print()
print(f"Accuracy Score for QDA: ", accuracy_score(y_test, y_pred_skqda))
print(f"Recall Score for QDA: ", recall_score(y_test, y_pred_skqda))
print(f"F1 Score for QDA: ", f1_score(y_test, y_pred_skqda))
print(classification_report(y_test, y_pred_skqda))


Sklearn :
Accuracy Score for LDA:  0.9
Recall Score for LDA:  0.8658536585365854
F1 Score for LDA:  0.8987341772151899
              precision    recall  f1-score   support

           0       0.87      0.94      0.90        78
           1       0.93      0.87      0.90        82

    accuracy                           0.90       160
   macro avg       0.90      0.90      0.90       160
weighted avg       0.90      0.90      0.90       160


Accuracy Score for QDA:  0.9375
Recall Score for QDA:  0.926829268292683
F1 Score for QDA:  0.9382716049382716
              precision    recall  f1-score   support

           0       0.93      0.95      0.94        78
           1       0.95      0.93      0.94        82

    accuracy                           0.94       160
   macro avg       0.94      0.94      0.94       160
weighted avg       0.94      0.94      0.94       160



In [24]:
print("Scratch :")
print(f"Accuracy Score for LDA: ", accuracy_score(y_test, y_pred_lda))
print(f"Recall Score for LDA: ", recall_score(y_test, y_pred_lda))
print(f"F1 Score for LDA: ", f1_score(y_test, y_pred_lda))
print(classification_report(y_test, y_pred_lda))

print()
print(f"Accuracy Score for QDA: ", accuracy_score(y_test, y_pred_qda))
print(f"Recall Score for QDA: ", recall_score(y_test, y_pred_qda))
print(f"F1 Score for QDA: ", f1_score(y_test, y_pred_qda))
print(classification_report(y_test, y_pred_qda))


Scratch :
Accuracy Score for LDA:  0.9625
Recall Score for LDA:  0.926829268292683
F1 Score for LDA:  0.9620253164556963
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        78
           1       1.00      0.93      0.96        82

    accuracy                           0.96       160
   macro avg       0.96      0.96      0.96       160
weighted avg       0.97      0.96      0.96       160


Accuracy Score for QDA:  0.81875
Recall Score for QDA:  0.6463414634146342
F1 Score for QDA:  0.7851851851851851
              precision    recall  f1-score   support

           0       0.73      1.00      0.84        78
           1       1.00      0.65      0.79        82

    accuracy                           0.82       160
   macro avg       0.86      0.82      0.81       160
weighted avg       0.87      0.82      0.81       160



Compute the ROC AUC Score using `sklearn` (for both LDA and QDA models trained using this library)

In [25]:
print("Sklearn:")

print(f"ROC AUC Score for LDA : ", roc_auc_score(y_test, y_pred_sklda))
print()
print(f"ROC AUC Score for QDA : ", roc_auc_score(y_test, y_pred_skqda))

Sklearn:
ROC AUC Score for LDA :  0.9008755472170106

ROC AUC Score for QDA :  0.9377736085053158


In [26]:
print("Scratch:")

print(f"ROC AUC Score for LDA : ", roc_auc_score(y_test, y_pred_lda))
print()
print(f"ROC AUC Score for QDA : ", roc_auc_score(y_test, y_pred_qda))

Scratch:
ROC AUC Score for LDA :  0.9634146341463414

ROC AUC Score for QDA :  0.8231707317073171


# Kaggle Competition

Link: (will be added soon)

In [27]:
def prepare_test_data(test_df,train_df):
    test_df = test_df.drop(['Ad Topic Line', 'Timestamp', 'ID'], axis=1)
    test_df = pd.get_dummies(test_df, columns=['City', 'Country'], drop_first=True, dtype=int)
    le = LabelEncoder()
    test_df['Gender'] = le.fit_transform(test_df['Gender'])
    test_df = test_df.reindex(columns=train_df.columns, fill_value=0)
    test_df = test_df[train_df.columns.tolist()]
    return test_df.to_numpy()

test_df = pd.read_csv("assignment2-test-data.csv")
test_df = prepare_test_data(test_df,features_df)
y_test_pred_lda = lda.predict(test_df)
new_pd_data = pd.DataFrame(y_pred_lda, columns=['Clicked on Ad'])
new_pd_data['ID'] = np.arange(0, len(new_pd_data))
new_pd_data.to_csv("lda_pred.csv", index=False)
new_pd_data.head(5)

,Clicked on Ad,ID
0,1,0
1,0,1
2,0,2
3,0,3
4,1,4
